# %% [markdown]
# # Task 01 — Exploratory Data Analysis (EDA)
#
# **Project:** KAIM Quant Forecast & Portfolio Optimization 2025  
# **Notebook:** 01_EDA.ipynb  
# **Purpose:** Load processed TSLA, BND, SPY daily data (2015-07-01 → 2025-07-31), perform statistical & visual EDA as per brief.  
# **Author:** [Your Name]
#
# ---
#
# ## Objectives (from project brief)
# - Check data quality (types, missing values, date alignment)
# - Compute descriptive stats (mean, volatility, skew, kurtosis)
# - Plot time series for adjusted close, returns, rolling volatility
# - Detect & quantify outliers in returns
# - Run Augmented Dickey-Fuller tests for stationarity
# - Compute risk metrics: historical VaR, Sharpe ratio
# - Document insights for each plot/test (investment-relevant interpretation)
#
# ## Why this matters:
# EDA is **decision insurance** — before modeling, we need to be sure our data is valid, aligned, and that we understand statistical properties that will affect forecasts and portfolio construction.


In [ ]:
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from statsmodels.tsa.stattools import adfuller
from scipy.stats import skew, kurtosis

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context('talk')

PROC_DIR = Path("../data/processed")


# %% [markdown]
# ## 2. Load processed data
# Data comes from `src/data/preprocess.py` — aligned on business days, with `*_adj` and `*_ret` columns.


In [ ]:
# %%
df = pd.read_csv(PROC_DIR / "combined_adj_and_returns.csv", index_col=0, parse_dates=True)
df.head()


# %% [markdown]
# **Insight:**  
# - Columns: `TSLA_adj`, `TSLA_ret`, `BND_adj`, `BND_ret`, `SPY_adj`, `SPY_ret`  
# - Frequency: business days.  
# - Return columns are simple daily returns.  
# - Price columns will be used for plotting long-term trends; return columns for volatility & risk metrics.


In [ ]:
# %%
missing_counts = df.isna().sum()
dtypes = df.dtypes
date_min, date_max = df.index.min(), df.index.max()

print("Missing values:\n", missing_counts)
print("\nData types:\n", dtypes)
print(f"\nDate range: {date_min} → {date_max}")


# %% [markdown]
# **Elite insight:**  
# - Any persistent NaNs in returns after preprocessing suggest corporate actions or missing market data — would require forward/backward fill decisions.  
# - Date alignment ensures covariance matrix in portfolio step is valid.  
# - Non-float types in price/returns columns would break model fitting.


In [ ]:
# %%
desc_stats = {}
for ticker in ["TSLA", "BND", "SPY"]:
    ret = df[f"{ticker}_ret"].dropna()
    desc_stats[ticker] = {
        "mean_daily": ret.mean(),
        "ann_mean": ret.mean()*252,
        "std_daily": ret.std(),
        "ann_vol": ret.std()*np.sqrt(252),
        "skew": skew(ret),
        "kurtosis": kurtosis(ret, fisher=True)
    }

pd.DataFrame(desc_stats).T


# %% [markdown]
# **Elite insight:**  
# - TSLA expected to have higher annualized volatility (>50%) vs BND (<10%).  
# - Skew/kurtosis tell us about fat tails & asymmetry — TSLA typically shows positive skew but heavy tails (kurtosis >> 3).  
# - BND’s low vol and near-normal distribution is bond-typical.


# %% [markdown]
# ## 5. Price series plots
# Visual inspection of price trends.


In [ ]:
# %%
fig, ax = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
for i, ticker in enumerate(["TSLA", "BND", "SPY"]):
    ax[i].plot(df.index, df[f"{ticker}_adj"], label=f"{ticker} Adj Close")
    ax[i].set_title(f"{ticker} Adjusted Close Price")
    ax[i].legend()
plt.tight_layout()
plt.show()


# %% [markdown]
# **Elite insight:**  
# - TSLA shows explosive growth periods & sharp drawdowns — relevant for model non-stationarity.  
# - BND stable, mean-reverting range; SPY upward trend with crises visible (e.g., COVID, 2022 rate hikes).


# %% [markdown]
# ## 6. Returns distribution
# Histograms + KDE for each asset’s daily returns.


In [ ]:
# %% [markdown]
# ## 7. Rolling volatility (30-day annualized)
# Helps identify regime changes in risk.


In [ ]:
# %%
window = 30
fig, ax = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
for i, ticker in enumerate(["TSLA", "BND", "SPY"]):
    vol = df[f"{ticker}_ret"].rolling(window).std()*np.sqrt(252)
    ax[i].plot(vol, label=f"{ticker} 30-day Annualized Vol")
    ax[i].set_title(f"{ticker} Rolling Volatility ({window} days)")
    ax[i].legend()
plt.tight_layout()
plt.show()


# %% [markdown]
# **Elite insight:**  
# - Volatility clusters — high vol persists; models may benefit from GARCH-like terms.  
# - TSLA volatility spikes align with market stress and company events (e.g., earnings).


# %% [markdown]
# ## 8. Outlier detection
# Identify top 5 absolute daily returns for each asset.


In [ ]:
# %%
outliers = {}
for ticker in ["TSLA", "BND", "SPY"]:
    abs_ret = df[f"{ticker}_ret"].abs()
    top5 = abs_ret.sort_values(ascending=False).head(5)
    outliers[ticker] = df.loc[top5.index, [f"{ticker}_ret"]]

outliers


# %% [markdown]
# **Elite insight:**  
# Outliers are often linked to macro events (Fed announcements, pandemics) or idiosyncratic shocks (TSLA earnings).  
# They influence VaR & model training; consider robust estimators.


# %% [markdown]
# ## 9. Augmented Dickey-Fuller (ADF) tests
# Test for stationarity in prices and returns.


In [ ]:
# %%
def adf_test(series):
    res = adfuller(series.dropna(), autolag='AIC')
    return {"ADF Stat": res[0], "p-value": res[1], "# Lags": res[2], "# Observations": res[3]}

adf_results = {}
for ticker in ["TSLA", "BND", "SPY"]:
    adf_results[f"{ticker}_price"] = adf_test(df[f"{ticker}_adj"])
    adf_results[f"{ticker}_return"] = adf_test(df[f"{ticker}_ret"])

pd.DataFrame(adf_results).T


# %% [markdown]
# **Elite insight:**  
# - Price series p-values > 0.05 → fail to reject unit root → non-stationary (expected).  
# - Return series p-values < 0.05 → reject unit root → stationary (required for ARIMA without extra differencing).


# %% [markdown]
# ## 10. Risk metrics — VaR & Sharpe
# Value at Risk (5%) and annualized Sharpe ratio for each asset.


In [ ]:
# %%
def var_historic(returns, level=0.05):
    return np.percentile(returns.dropna(), 100*level)

def sharpe_ratio(returns, risk_free=0.0):
    ann_ret = returns.mean()*252
    ann_vol = returns.std()*np.sqrt(252)
    return (ann_ret - risk_free) / ann_vol

risk_metrics = {}
for ticker in ["TSLA", "BND", "SPY"]:
    ret = df[f"{ticker}_ret"]
    risk_metrics[ticker] = {
        "VaR_5%": var_historic(ret, 0.05),
        "Sharpe": sharpe_ratio(ret)
    }

pd.DataFrame(risk_metrics).T


# %% [markdown]
# **Elite insight:**  
# - TSLA’s VaR (5%) may be several times larger than BND’s — risk budgeting must account for position sizing.  
# - Sharpe >1 for SPY historically — robust core holding; TSLA’s Sharpe depends heavily on time period.


In [ ]:
# %% [markdown]
# ## 11. Summary Table for Memo
# Consolidate key metrics into one table for the Investment Memo appendix.


In [ ]:
# %%
summary_df = pd.concat([
    pd.DataFrame(desc_stats).T[["ann_mean", "ann_vol"]],
    pd.DataFrame(risk_metrics).T
], axis=1)
summary_df.to_csv("../reports/EDA_summary_metrics.csv")
summary_df


# %% [markdown]
# **Elite insight:**  
# - This table can be inserted directly into the Investment Memo appendix.  
# - For portfolio optimization, `ann_mean` and `ann_vol` provide intuition, but expected returns for TSLA will come from forecast model (Task 4).


# %% [markdown]
# ## 12. Next steps (from EDA to modeling)
# - Confirm returns stationarity for ARIMA.  
# - Consider volatility modeling (GARCH) if volatility clustering is strong.  
# - Remove/flag outliers if they unduly influence model fit.  
# - Proceed to Task 2 modeling with cleaned returns & prices.
#
# **EDA complete.**
